# Seminar I Intelligent Systems 

In [143]:
import pygad
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", message="The 'delay_after_gen' parameter is deprecated")

In [144]:
df = pd.read_json('hard_dataset_1.json')

num_papers = df.loc[0,"num_papers"]
df = df.drop(columns="num_papers")
print("Number of papers", num_papers)

num_reviewers = df.loc[0,"num_reviewers"]
df = df.drop(columns="num_reviewers")
print("Number of reviewers", num_reviewers)

reviewer_capacity = df.loc[0,"reviewer_capacity"]
df = df.drop(columns ="reviewer_capacity")
print("Number of reviewer capacity", reviewer_capacity)

min_reviews_per_paper = df.loc[0,"min_reviews_per_paper"]
df = df.drop(columns="min_reviews_per_paper")
print("Minimum reviewers per paper", min_reviews_per_paper)

max_reviews_per_paper = df.loc[0,"max_reviews_per_paper"]
df = df.drop(columns="max_reviews_per_paper")
print("Maximum reviewers per paper", max_reviews_per_paper)

df = df.drop(columns="dataset_id")

# print("Data", df)

preferences_matrix = np.array(df["preferences"].tolist())
friendships_matrix = np.array(df["friendships"].tolist())
authorship_matrix = np.array(df["authorship"].tolist())

penalties = [
    5, # penalty_reviewer_cap
    7, # penalty_low_reviewcount
    3, # penalty_high_reviewcount
    np.array([[0, 1, 2, 3, 4, 5], [3, 2, 1, 0, -1, -2]]), # preference_points
    7, # penalty_friendship
    9, # penalty_authorship 
]

configurations = [
    {"Parent Selection": "rws", "Generations": 10, "Parents Mating": 2, "Sol Per Pop": 10},
    {"Parent Selection": "rws", "Generations": 20, "Parents Mating": 2, "Sol Per Pop": 10},
    {"Parent Selection": "rws", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 10},
    {"Parent Selection": "rws", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 20},
    {"Parent Selection": "rws", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 50},
    {"Parent Selection": "rank", "Generations": 10, "Parents Mating": 2, "Sol Per Pop": 10},
    {"Parent Selection": "rank", "Generations": 20, "Parents Mating": 2, "Sol Per Pop": 10},
    {"Parent Selection": "rank", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 10},
    {"Parent Selection": "rank", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 20},
    {"Parent Selection": "rank", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 50},
    {"Parent Selection": "tournament", "Generations": 10, "Parents Mating": 2, "Sol Per Pop": 10},
    {"Parent Selection": "tournament", "Generations": 20, "Parents Mating": 2, "Sol Per Pop": 10},
    {"Parent Selection": "tournament", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 10},
    {"Parent Selection": "tournament", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 20},
    {"Parent Selection": "tournament", "Generations": 20, "Parents Mating": 4, "Sol Per Pop": 50},
]

results = []

for config in configurations:
    ga_instance = pygad.GA(
        fitness_func = fitness_function,
        parent_selection_type=config["Parent Selection"],
        crossover_type = custom_crossover,
        # crossover_probability=config.get("Crossover Probability", 0.8),  # Default 80%
        mutation_type = custom_mutation,
        # mutation_probability=config.get("Mutation Probability", 0.1), # Default 10%
        # oder mutation_percent_genes=config.get("Mutation Percent Genes", 10),
        num_generations = config["Generations"],
        num_parents_mating = config["Parents Mating"],
        sol_per_pop = config["Sol Per Pop"],
        initial_population = custom_initialpop(config["Sol Per Pop"], num_reviewers, num_papers),
        gene_type = int,
        num_genes = num_reviewers * num_papers,
        gene_space = [0, 1],
    )
    
    ga_instance.run()
    
    best_solution, best_solution_fitness, _ = ga_instance.best_solution()
    best_solution_matrix = best_solution.reshape(num_reviewers, num_papers)

    results.append({
        "Parent Selection": config["Parent Selection"],
        "Generations": config["Generations"],
        "Parents Mating": config["Parents Mating"],
        "Sol Per Pop": config["Sol Per Pop"],
        "Best Fitness": best_solution_fitness,
    })

df_results = pd.DataFrame(results)

print(df_results)

Number of papers 15
Number of reviewers 10
Number of reviewer capacity 6
Minimum reviewers per paper 3
Maximum reviewers per paper 5
   Parent Selection  Generations  Parents Mating  Sol Per Pop  Best Fitness
0               rws           10               2           10        -31.87
1               rws           20               2           10        -30.33
2               rws           20               4           10        -30.40
3               rws           20               4           20        -29.40
4               rws           20               4           50        -31.60
5              rank           10               2           10        -32.60
6              rank           20               2           10        -29.53
7              rank           20               4           10        -30.07
8              rank           20               4           20        -31.27
9              rank           20               4           50        -31.40
10       tournament           1

## Fitness

In [145]:
def is_author(reviewer, paper, authorship_matrix):

    # Checks if the reviewer is the author of the paper
    return authorship_matrix[reviewer, paper] == 1

def is_friend(reviewer, paper, authorship_matrix, friendships_matrix):
    
    # Get all authors of the paper
    authors_of_paper = np.where(authorship_matrix[:, paper] == 1)[0]
    
    # Checks if the reviewer is a friend of any author of the paper
    return np.any(friendships_matrix[authors_of_paper, reviewer] == 1)

In [146]:
# Compute fitness value with absolute or relative penalties
def f(assign_matrix, preferences_matrix, friendships_matrix, authorship_matrix, penalties): 
    
    fit_val = 0  
   
    # Penalties 
    penalty_reviewer_cap = penalties[0]       
    penalty_low_reviewcount = penalties[1]    
    penalty_high_reviewcount = penalties[2]   
    preference_points = penalties[3]          
    penalty_friendship = penalties[4]         
    penalty_authorship = penalties[5]         
    
    # Iterate over reviewers and papers
    for i in range(len(assign_matrix[:, 0])):  
        for j in range(len(assign_matrix[0, :])):  

            # Add preference penalty if the reviewer is assigned to the paper
            if assign_matrix[i, j] == 1: 
                index = preferences_matrix[i, j]
                fit_val += preference_points[1, index]

            # Add friendship penalty if the reviewer is a friend of any author of the paper
            if is_friend(i, j, authorship_matrix, friendships_matrix):
                fit_val += penalty_friendship 

            # Add authorship penalty if the reviewer is also the author of the paper
            if assign_matrix[i, j] == 1 and is_author(i, j, authorship_matrix):
                fit_val += penalty_authorship         
        
    # Check for reviewer capacity violations             
    for t in range(len(assign_matrix[:, 0])):  

        x = np.sum(assign_matrix[t, :])  #

        if x > reviewer_capacity:
            fit_val += penalty_reviewer_cap 
    
    # Check for review count violations 
    for z in range(len(assign_matrix[0, :])): 

        x = np.sum(assign_matrix[:, z]) 

        if x < min_reviews_per_paper:  
            fit_val += (min_reviews_per_paper - x) * penalty_low_reviewcount
            
        if x > max_reviews_per_paper:  
            fit_val += (x - max_reviews_per_paper) * penalty_high_reviewcount

    # Normalize fitness value 
    fit_val = round((-1 * fit_val / (num_reviewers * num_papers) * 10), 2)
    return fit_val

In [147]:
def fitness_function(ga_instance, solution, solution_idx):

    solution = solution.reshape(num_reviewers,num_papers)

    return f(solution, preferences_matrix, friendships_matrix, authorship_matrix, penalties)   

In our fitness function, we evaluate reviewer-paper assignments by balancing rewards for high preferences with penalties for constraint violations. We discourage conflicts of interest by penalizing assignments where reviewers are friends with authors or are the authors themselves.

To maintain fairness, we enforce reviewer capacity limits and ensure each paper meets the required number of reviewers, applying penalties for excesses or deficits.

Finally, we normalize the fitness score by the total number of assignments, making it consistent.

## Crossover

In [148]:
def is_author_or_friend(reviewer, paper): 

    # Check if the reviewer is the author of the paper
    if authorship_matrix[reviewer, paper] == 1:
        return True

    # Get all authors of the paper
    authors_of_paper = np.where(authorship_matrix[:, paper] == 1)[0]
    
    # Check if the reviewer is a friend of any of the paper's authors
    if np.any(friendships_matrix[authors_of_paper, reviewer] == 1):
        return True

    return False

In [149]:
# Crossover function that returns offspring which fullfills the constraints 
def custom_crossover(parents, offspring_size, ga_instance):
    
    offspring = []

    # Generate offspring
    for i in range(offspring_size[0]):

        # Select two parents for crossover
        parent1 = parents[np.random.randint(0, parents.shape[0]), :].copy()
        parent2 = parents[np.random.randint(0, parents.shape[0]), :].copy()

        # Reshape parents 
        parent1 = parent1.reshape(num_reviewers, num_papers)
        parent2 = parent2.reshape(num_reviewers, num_papers)

        # Randomly split columns (papers) between the two parents
        hereditary_parent1 = num_papers // 2
        indices_parent1 = np.random.choice(range(num_papers), size=hereditary_parent1, replace=False)
        indices_parent2 = [j for j in range(num_papers) if j not in indices_parent1]

        # Create a new child by combining parts of both parents
        new_child = np.zeros_like(parent1)
        new_child[:, indices_parent1] = parent1[:, indices_parent1]
        new_child[:, indices_parent2] = parent2[:, indices_parent2]
        
        offspring.append(new_child)

    offspring = np.array(offspring)
    
    # Check if reviewer capacity is violated and that remove papers in preference order
    for i in range(offspring.shape[0]): 
        for j in range(offspring.shape[1]):  

            num_assigned = np.sum(offspring[i, j, :])  

            if num_assigned > reviewer_capacity:
                to_remove = num_assigned - reviewer_capacity               
                idx_assigned = np.where(offspring[i, j, :] == 1)[0]              
                preferences = preferences_matrix[j, idx_assigned]               
                sorted_indices = np.argsort(preferences) 
                lowest_preference_indices = sorted_indices[:to_remove]
                indices_to_remove = idx_assigned[lowest_preference_indices]

                offspring[i, j, indices_to_remove] = 0
                    
    # Add reviewer assignment to fullfill review_requirements, only add assignment if no other constraints are violated
    for i in range(offspring.shape[0]):  
        for j in range(offspring.shape[2]):

            if np.sum(offspring[i,:,j]) < min_reviews_per_paper: 
                available_reviewers = np.where((offspring[i,:,j] == 0) & (np.sum(offspring[i,:,:], axis=1) < reviewer_capacity))[0]

                for reviewer in available_reviewers: 

                    if (is_author_or_friend(reviewer, j) == False): 
                        offspring[i,reviewer,j] = 1
                        break

    # TODO Edge case: no valid reviewer is found?

    return offspring.reshape(offspring.shape[0], offspring.shape[1]*offspring.shape[2])       

In our crossover function, we generate offspring by combining assignments from two parents while maintaining constraints. The function ensures a balanced distribution of papers between parents and avoids exceeding reviewer capacity by removing assignments with the lowest preferences when necessary.

To meet review requirements, we add assignments only if they do not violate constraints, such as reviewer capacity or conflicts of interest. The resulting offspring respects all constraints.

## Mutation

In [ ]:
# Mutation function that modifies offspring while respecting constraints
def custom_mutation(offspring, ga_instance):

    # Reshape offspring for easier handling
    offspring = offspring.reshape(offspring.shape[0], num_reviewers, num_papers)
    num_mutations = (np.round(num_reviewers * num_papers * 0.05)).astype(int)
    
    for i in range(offspring.shape[0]):  # Iterate over each child
        for _ in range(num_mutations):  # Apply mutations
            
            # Attempt to add an assignment
            idx_assign, random_col = -1, -1
            cols = np.arange(0, num_papers)

            while len(cols) > 0:
                random_col = np.random.choice(range(num_papers), size=1, replace=False)
                idxs_not_assigned = np.where(offspring[i, :, random_col] == 0)[1]
                is_valid = False 

                while len(idxs_not_assigned) > 0: 
                    idx_assign = np.random.choice(idxs_not_assigned, size=1)[0]
                    if not is_author_or_friend(idx_assign, random_col):
                        offspring[i, idx_assign, random_col] = 1
                        is_valid = True
                        break
                    else:
                        idxs_not_assigned = np.delete(idxs_not_assigned, np.where(idxs_not_assigned == idx_assign))
                              
                if is_valid: 
                    break
                else: 
                    cols = np.delete(cols, np.where(cols == random_col)[0])

            # TODO Edge case: no valid reviewer is found?

            # Handle capacity violations by removing excess assignments
            if np.sum(offspring[i, idx_assign, :]) > reviewer_capacity:
                reviewer_assignments = np.where(offspring[i, idx_assign, :] == 1)[0]
                reviewer_assignments = np.delete(reviewer_assignments, np.where(reviewer_assignments == random_col))
                random_paper = np.random.choice(reviewer_assignments, size=1)[0]
                offspring[i, idx_assign, random_paper] = 0
            else:                
                all_assigned = np.array(np.where(offspring[i, :, :] == 1)).T
                valid_points = [point for point in all_assigned if point[0] != idx_assign or point[1] != random_col]
                if len(valid_points) > 0:
                    random_point = valid_points[np.random.choice(len(valid_points))]
                    offspring[i, random_point[0], random_point[1]] = 0

    return offspring.reshape(offspring.shape[0], num_reviewers * num_papers)


In our mutation function, we introduce diversity into the population by modifying assignments while adhering to constraints. Assignments are added only if they do not conflict with authorship or friendship constraints.

To maintain reviewer capacity limits, excess assignments are removed, not prioritizing low-preference reviews, by keeping it random.

## Initial population

In [151]:
# Create initial position with random values  
def custom_initialpop(num_solutions, num_reviewers, num_papers):

    pop = np.random.randint(0, 2, size=(num_solutions, num_reviewers, num_papers))
    return pop.reshape((num_solutions, -1))